<center><img src="../图片数据/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

# 远程手柄遥控
在本例中，我们将使用连接到web浏览器机器的gamepad控制器远程控制Jetbot。

### 创建手柄控制器
我们要做的第一件事是创建一个' Controller'widget的实例，我们将使用它来驱动我们的Jetbot。
“Controller”小部件接受一个“index”参数，该参数指定控制器的数量。如果你有多个控制器，或者一些游戏手柄以多个控制器的形式出现，这是非常有用的。想要正确的使用你的手柄来控制
Jetbot你得:
1. 打开[http://html5gamepad.com](http://html5gamepad.com)此网页.  
2. 按下你正在使用的手柄的按键
3. 记住当你按下按键后弹出的相应的索引号

接下来，我们将使用该索引创建并显示控制器。

In [ ]:
import ipywidgets.widgets as widgets
controller = widgets.Controller(index=0)  #用你刚测试过正在使用的控制器的索引号替代
display(controller)

相关模块导入

In [ ]:
#函数库路径导入
from servoserial import ServoSerial
from jetbot import Robot
import traitlets
from jetbot import Camera
from jetbot import bgr8_to_jpeg
from jetbot import Heartbeat
import threading
import time
# 线程功能操作库
import inspect
import ctypes

import traitlets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg
import os
from uuid import uuid1

创建并打开显示相机初始化实例

In [ ]:
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

添加心跳连接

In [ ]:
def handle_heartbeat_status(change):
    if change['new'] == Heartbeat.Status.dead:
        camera_link.unlink()
        robot.stop()

heartbeat = Heartbeat(period=0.5)
# 将回调函数附加到heartbeat状态
heartbeat.observe(handle_heartbeat_status, names='status')

创建主动停止进程的方法

In [ ]:
def _async_raise(tid, exctype):
    """raises the exception, performs cleanup if needed"""
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        # """if it returns a number greater than one, you're in trouble,
        # and you should call it again with exc=NULL to revert the effect"""
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        
def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

分别创建云台相机上、下、左、右运动的方法

In [ ]:
'''以下是舵机控制的部分    '''
servo_device = ServoSerial() 
def camUpFunction():
    global updownpulse
    updownpulse+=15
    if updownpulse>4095:
        updownpulse=4095
    servo_device.Servo_serial_control(2, updownpulse)

def camDownFunction():
    global updownpulse
    updownpulse-=15
    if updownpulse<1300:
        updownpulse=1300
    servo_device.Servo_serial_control(2, updownpulse)

def camLeftFunction():
    global leftrightpulse
    leftrightpulse+=15
    if leftrightpulse>3600:
        leftrightpulse=3600
    servo_device.Servo_serial_control(1, leftrightpulse)

def camRightFunction():
    global leftrightpulse
    leftrightpulse-=15
    if leftrightpulse<600:
        leftrightpulse=600
    servo_device.Servo_serial_control(1, leftrightpulse)

def camservoInitFunction():
    global leftrightpulse, updownpulse
    leftrightpulse = 2048
    updownpulse = 2048
    servo_device.Servo_serial_control(1, 2048)
    time.sleep(0.1)
    servo_device.Servo_serial_control(2, 2048)

### 初始化云台相机位置
运行下面单元格代码初始化云台项目至初始位置

In [ ]:
camservoInitFunction()

### 加载Robot类
准备开始为JetBot编程前，我们需要导入“Robot”类。这个类允许我们轻松控制JetBot的电机

In [ ]:
robot = Robot()

### 创建板载呼吸灯方法

In [ ]:
def BLN_Onboard():
    global i , k
    i = k = 0
    while True:
        if k == 0:
            robot.set_bln(i)
            i += 0.01
            if( i >= 1 ):
                k = 1
            time.sleep(0.005)
        elif k == 1:
            robot.set_bln(i)
            i -=0.01
            if i <= 0 :
                k = 0
            time.sleep(0.005)

通过运行下面单元格代码开启板载呼吸灯独立进程

In [ ]:
thread1 = threading.Thread(target=BLN_Onboard)
# thread1.setDaemon(True)
thread1.start()

<center><img src="../图片数据/手柄.png" alt="Header" style="width: 400px;"/></center>

# 创建手柄遥杆实时控制Jetbot运动的方法  
### 使用前请先通过手柄中间的ANOLOG键按一下转换至模拟模式

程序功能: 
         1. 做遥杆控制Jetbot移动,右摇杆控制舵机移动  
         2. 按下SELECT键将云台角度复位  
         3. 按下L侧1键控制云台上升,按下L侧2键控制云台下降  
    上图所示为Jetbot Yahboom配套的手柄实物参考图,在之后的默认代码中默认使用这一款手柄,别的手柄的请参考键位表改值
    如果使用的成本较低的Jetbot Yahboom配套的手柄的话请使用代码段---1
    如果使用的是Xbox360的手柄的话请使用代码段---2

In [ ]:
def jetbot_motion():
    count1 = count2 = count3 = count4 =  count5 = 0
    while 1:
        #小车左右DC motor
        #手柄操作代码---1(Jetbot Yahboom配套的手柄)
        #因为Jetbot配套使用的为非品牌的USB Gamepad,所有在遥杆复位值是0.0039,为了防止未操作时电机异响加了以下代码进行操作
        if controller.axes[1].value <= 0.1:
            if (controller.axes[0].value <= 0.1 and controller.axes[0].value >= -0.1 
                and controller.axes[1].value <= 0.1 and controller.axes[1].value >= -0.1):
                robot.stop()
            else:
                robot.set_motors(-controller.axes[1].value + controller.axes[0].value, -controller.axes[1].value - controller.axes[0].value)
            
            time.sleep(0.01)
        else:
            robot.set_motors(-controller.axes[1].value - controller.axes[0].value, -controller.axes[1].value + controller.axes[0].value)
            time.sleep(0.01)
          #手柄操作代码---2(Xbox360手柄)
#         if controller.axes[1].value <= 0:
#             robot.set_motors(-controller.axes[1].value + controller.axes[0].value, -controller.axes[1].value - controller.axes[0].value)
#             time.sleep(0.01)
#         else:
#             robot.set_motors(-controller.axes[1].value - controller.axes[0].value, -controller.axes[1].value + controller.axes[0].value)
#             time.sleep(0.01)

        #镜头上下舵机
        if controller.axes[2].value == 1:
            count1  += 1
            if count1 >= 3:
                camDownFunction()
                count1 = 0
        elif controller.axes[2].value == -1:
            count1  += 1
            if count1 >= 3:
                camUpFunction()
                count1 = 0
        else:
            count1 = 0
        #镜头左右舵机    
        if controller.axes[5].value == 1:
            count2  += 1
            if count2 >= 3:
                camRightFunction()
                count2 = 0
        elif controller.axes[5].value == -1:
            count2  += 1
            if count2 >= 3:
                camLeftFunction()
                count2 = 0
        else:
            count2 = 0
        #镜头上下左右舵机 复位
        if controller.buttons[8].value == True:
            count3 += 1
            if count3 >= 3:
                camservoInitFunction()
                count3 = 0
        else:
            count3 = 0
        
        #镜头升降
        if controller.buttons[6].value == True:
            count4 += 1
            if count4 >= 3:
                robot.set_vertical_motors(1)
                count4 = 0
        elif controller.buttons[4].value == True:
            count4 += 1
            if count4 >= 3:
                robot.set_vertical_motors(-1)
                count4 = 0
        else:
            robot.set_vertical_motors(0)
            count4 = 0

通过运行下面单元格代码开启手柄实时控制Jetbot运动的独立进程

In [ ]:
thread2 = threading.Thread(target=jetbot_motion)
thread2.setDaemon(True)
thread2.start()

通过运行下面单元格代码结束板载呼吸灯独立进程

In [ ]:
stop_thread(thread1)

通过运行下面单元格代码结束手柄实时控制Jetbot运动的独立进程

In [ ]:
stop_thread(thread2)
robot.stop()